In [ ]:
import pandas as pd
import seaborn as sns

data = pd.read_csv("./heart.csv", sep=",")

In [ ]:
import pandas as pd
import matplotlib as plb
import matplotlib.pyplot as plt
from sklearn import preprocessing
import imblearn
from collections import Counter

def pre(data): 
    to_drop_columns = [ 'exang', 'ca', 'cp', 'slope', 'oldpeak', 'restecg']
    data.drop(to_drop_columns, axis=1, inplace=True)
    colonne_tenute = ["age","sex","trestbps","chol","fbs","thalach","thal", "target"]
    
    scaler = preprocessing.MinMaxScaler((0, 1))
    data = scaler.fit_transform(data)
    data = pd.DataFrame(data=data, columns=colonne_tenute)
    print(data)

    oversample = imblearn.over_sampling.SMOTE() #usa k-nearest-neighbours
    x_data, y_data = oversample.fit_resample(data.iloc[: , : -1], data["target"])


    return x_data, y_data
        

In [ ]:
from sklearn import naive_bayes
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute

x_train, y_train = pre(data)
flag = 0

for i in range (0, 50):
    cv = KFold(n_splits=10)
    model = naive_bayes.GaussianNB()
    score = cross_val_score(model, x_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1) #errore assoluto per 10 split
    flag = flag + mean(absolute(score))
print(flag/50)
#fa 10 volte il training e stampa la media dell'errore assoluto di ogni validazione


In [ ]:

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


from pyJoules.energy_meter import EnergyContext
from pyJoules.device.rapl_device import RaplPackageDomain, RaplDramDomain
from pyJoules.handler.csv_handler import CSVHandler
csv_handler = CSVHandler('bayes_results_training.csv')

model = naive_bayes.GaussianNB()

with EnergyContext(handler=csv_handler, 
                    domains=[RaplPackageDomain(0), 
                            RaplDramDomain(0)]) as ctx:
    model.fit(x_train, y_train)

csv_handler.save_data()


csv_handler = CSVHandler('bayes_results_prediction.csv')
with EnergyContext(handler=csv_handler, 
                    domains=[RaplPackageDomain(0), 
                            RaplDramDomain(0)]) as ctx:
    prediction = model.predict(x_train[701:])

csv_handler.save_data()




labels = ['positivo', 'negativo']
matrice = confusion_matrix(y_train[701:], prediction)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(matrice)
plt.title('Matrice di Confusione \n Reali')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.ylabel('Predetti')
#plt.xlabel('Reali')
plt.show()


recall_value = recall_score(y_train[701:], prediction)
print("recall = ", recall_value)

precision_value = precision_score(y_train[701:], prediction)
print("precision = ", precision_value)

accuracy_value = accuracy_score(y_train[701:], prediction)
print("accuracy = ", accuracy_value)

fmeasure_value = f1_score(y_train[701:], prediction)
print("f1 = ", fmeasure_value)

In [ ]:
import flask
from flask import request, jsonify
from flask import Response
import numpy as np
import pandas as pd

app = flask.Flask(__name__)

@app.route('/', methods=['POST'])
def predici():
    query_parameters = request.get_json()
    x_data = pd.DataFrame(query_parameters, index=[0])
    
    age = query_parameters["age"]
    sex = query_parameters["sex"]
    trestbps = query_parameters["trestbps"]
    chol = query_parameters["chol"]
    fbs = query_parameters["fbs"]
    thalach = query_parameters["thalach"]
    thal = query_parameters["thal"]

    v = np.array(x_data)

    scaler = preprocessing.MinMaxScaler((0, 1))
    x_data = scaler.fit_transform(v.reshape(7,1))


    colonne_tenute = ["age","sex","trestbps","chol","fbs","thalach","thal"]
    x_data = pd.DataFrame(data=x_data.reshape(1, 7), columns=colonne_tenute)

    print("dopo scaling ", x_data)
    
    #chiama il modello
    csv_handler = CSVHandler('bayes_results_prediction.csv')

    with EnergyContext(handler=csv_handler, 
                    domains=[RaplPackageDomain(0), 
                            RaplDramDomain(0)]) as ctx:
        ctx.record()
        result = model.predict(x_data)

    csv_handler.save_data()


    return str(result)

if __name__ == "__main__":
    app.run(port=8082)